In [1]:
import os
import cv2
import dlib

def create_folders(crop_folder):
    """크롭 폴더 구조를 생성합니다."""
    os.makedirs(crop_folder, exist_ok=True)

def pad_image(frame):
    """이미지에 패딩을 추가합니다."""
    orig_height, orig_width = frame.shape[:2]
    max_dim = max(orig_height, orig_width)
    top_pad = (max_dim - orig_height) // 2
    bottom_pad = max_dim - orig_height - top_pad
    left_pad = (max_dim - orig_width) // 2
    right_pad = max_dim - orig_width - left_pad

    padded_frame = cv2.copyMakeBorder(frame, top_pad, bottom_pad, left_pad, right_pad, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    return padded_frame


def video_capture_labeling(video_path, class_id, crop_folder):
    try:
        # 비디오 캡처 객체 생성
        cap = cv2.VideoCapture(video_path)

        # 성공적으로 동영상을 열었는지 확인
        if not cap.isOpened():
            raise FileNotFoundError(f"Error: Could not open video file {video_path}")

        # FPS 정보 가져오기
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps == 0:
            raise ValueError("Error: FPS value is 0. Check the video file.")

        # 동영상의 총 프레임 수 가져오기
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # 동영상의 총 길이 (초) 계산
        total_duration = total_frames / fps
        print(f"Total duration: {total_duration} seconds")

        # 캡처 간격 (초)
        interval_sec = 0.1

        # 동영상 길이에 따른 캡처할 프레임 번호 리스트 생성
        frame_numbers = [int(fps * t) for t in [i * interval_sec for i in range(int(total_duration / interval_sec))]]

        print(f"Frame numbers to capture: {frame_numbers}")

        # dlib의 강아지 얼굴 탐지 모델 로드
        detector = dlib.cnn_face_detection_model_v1('dogHeadDetector.dat')

        capture_count = 1

        for frame_number in frame_numbers:
            # 원하는 프레임으로 이동
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)

            # 프레임 읽기
            ret, frame = cap.read()

            # 프레임이 성공적으로 읽혔는지 확인
            if ret:
                # 강아지 얼굴 탐지
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                detections = detector(gray, 1)
                
                # 얼굴이 검출된 경우만 저장
                if detections:
                    orig_height, orig_width = frame.shape[:2]

                    for i, d in enumerate(detections):
                        left, top, right, bottom, _ = (d.rect.left(), d.rect.top(), d.rect.right(), d.rect.bottom(), d.confidence)

                        # 좌표가 이미지 범위를 벗어나지 않도록 조정
                        left = max(0, left)
                        top = max(0, top)
                        right = min(orig_width, right)
                        bottom = min(orig_height, bottom)

                        # 박스에 맞게 이미지 크롭
                        cropped_img = frame[top:bottom, left:right]

                        # 크롭된 이미지가 유효한지 확인
                        if cropped_img.size == 0:
                            print(f"Warning: Cropped image is empty. Skipping this detection.")
                            continue

                        # 크롭된 이미지 리사이즈 (224x224)
                        resized_cropped_img = cv2.resize(cropped_img, (224, 224))

                        # 개별 동영상 폴더 생성
                        video_folder_name = os.path.splitext(os.path.basename(video_path))[0]
                        video_crop_folder = os.path.join(crop_folder, video_folder_name)
                        os.makedirs(video_crop_folder, exist_ok=True)

                        # 크롭된 이미지 저장 경로 설정
                        crop_image_name = f'dog{class_id}_{capture_count}.jpg'
                        crop_image_path = os.path.join(video_crop_folder, crop_image_name)

                        # 크롭된 이미지 저장
                        cv2.imwrite(crop_image_path, resized_cropped_img)
                        print(f"Cropped and resized image saved as {crop_image_path}")

                        capture_count += 1

    except FileNotFoundError as e:
        print(e)
    except ValueError as e:
        print(e)
    except RuntimeError as e:
        print(e)
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        if 'cap' in locals():
            # 캡처 객체 해제
            cap.release()

def extract_class_id_from_filename(filename):
    """확장자를 제거하고 파일명에서 숫자를 추출합니다."""
    basename = os.path.splitext(filename)[0]
    numbers = ''.join(filter(str.isdigit, basename))
    return int(numbers) if numbers else 0

def process_all_videos_in_folder(folder_path, crop_folder):
    """폴더 내 모든 비디오 파일을 처리합니다."""
    create_folders(crop_folder)
    
    files = os.listdir(folder_path)
    video_files = [f for f in files if f.endswith('.mp4')]
    
    for video_file in video_files:
        video_path = os.path.join(folder_path, video_file)
        class_id = extract_class_id_from_filename(video_file)
        print(f"Processing {video_path} with class ID {class_id}...")
        video_capture_labeling(video_path, class_id, crop_folder)

folder_path = '../sample_video'
crop_folder = '../cropped_images'

process_all_videos_in_folder(folder_path, crop_folder)


Processing ../sample_video\dog0.mp4 with class ID 0...
Total duration: 18.685333333333332 seconds
Frame numbers to capture: [0, 2, 4, 7, 9, 11, 14, 16, 19, 21, 23, 26, 28, 31, 33, 35, 38, 40, 43, 45, 47, 50, 52, 55, 57, 59, 62, 64, 67, 69, 71, 74, 76, 79, 81, 83, 86, 88, 91, 93, 95, 98, 100, 103, 105, 107, 110, 112, 115, 117, 119, 122, 124, 127, 129, 131, 134, 136, 139, 141, 143, 146, 148, 151, 153, 155, 158, 160, 163, 165, 167, 170, 172, 175, 177, 179, 182, 184, 187, 189, 191, 194, 196, 199, 201, 203, 206, 208, 210, 213, 215, 218, 220, 222, 225, 227, 230, 232, 234, 237, 239, 242, 244, 246, 249, 251, 254, 256, 258, 261, 263, 266, 268, 270, 273, 275, 278, 280, 282, 285, 287, 290, 292, 294, 297, 299, 302, 304, 306, 309, 311, 314, 316, 318, 321, 323, 326, 328, 330, 333, 335, 338, 340, 342, 345, 347, 350, 352, 354, 357, 359, 362, 364, 366, 369, 371, 374, 376, 378, 381, 383, 386, 388, 390, 393, 395, 398, 400, 402, 405, 407, 409, 412, 414, 417, 419, 421, 424, 426, 429, 431, 433, 436, 438, 44